In [1]:
from viresclient import SwarmRequest
from viresclient import ClientConfig
import datetime as dt
import sys,os
from pathlib import Path
import pandas as pd
import numpy as np
import datetime, time
from datetime import timedelta
import math
import pathlib
from datetime import datetime
import time
import calendar
import datetime

import chaosmagpy as cp
from chaosmagpy import load_CHAOS_matfile
from chaosmagpy.data_utils import mjd2000
from gg_to_geo import gg_to_geo

In [2]:
VirESToken = "rp-foC8gz-VTFz7He6SMPnQtKEpasEaB"

## Read the GPS Track

In [110]:
#Enter the name of your GPS track csv file including the extension .csv  and press Enter (e.g. BirdGPSTrajectory.csv)
gpsfilename=input()
Lat=input()
Long=input()
DateTime=input()

LER_22_23_Jun_15.csv
Lat
Long
DateTime


In [111]:
os.chdir(r"../data")
def getGPSData():
    nfp = pd.read_csv(gpsfilename, parse_dates=[0], encoding='utf-8', dayfirst=True, usecols=[Lat, Long, DateTime])
    return nfp

In [112]:
GPSData = getGPSData()
GPSData.rename(columns={Lat: 'gpsLat', Long: 'gpsLong', DateTime: 'gpsDateTime'}, inplace = True)
GPSData

,gpsLat,gpsLong,gpsDateTime
0,60.13,-1.18,22/06/2015 00:00
1,60.13,-1.18,22/06/2015 00:01
2,60.13,-1.18,22/06/2015 00:02
3,60.13,-1.18,22/06/2015 00:03
4,60.13,-1.18,22/06/2015 00:04
...,...,...,...
2875,60.13,-1.18,23/06/2015 23:55
2876,60.13,-1.18,23/06/2015 23:56
2877,60.13,-1.18,23/06/2015 23:57
2878,60.13,-1.18,23/06/2015 23:58


In [113]:
#2.1 Convert the gpsDateTime to datetime python object
GPSData['gpsDateTime'] = pd.to_datetime(GPSData['gpsDateTime'], format='%d/%m/%Y %H:%M')
GPSData['gpsDateTime'] = GPSData['gpsDateTime'].map(lambda x: x.replace(second=0))
GPSData['gpsLat'] = GPSData['gpsLat'].astype(float)
#2.2 Adding new column epoch, will be usefuel to compare the date&time o each gps point againts the gathered swmarm data points
GPSData['epoch'] = GPSData['gpsDateTime'].astype('int64')//1e9
GPSData['epoch'] = GPSData['epoch'].astype(int)
#2.3 Computing Date and Time columns
GPSData['dates'] = GPSData['gpsDateTime'].dt.date
GPSData['times'] = GPSData['gpsDateTime'].dt.time
#2.4 Maximum and Minimum values
MinDateTime = min(GPSData['gpsDateTime'])
MaxDateTime = max(GPSData['gpsDateTime'])
dt_object1 = MinDateTime.to_pydatetime()
dt_object2 = MaxDateTime.to_pydatetime()
startDate = MinDateTime.date()
endDate = MaxDateTime.date()
MinTime=MinDateTime.time()
MaxTime=MaxDateTime.time()
GPSData.head()

,gpsLat,gpsLong,gpsDateTime,epoch,dates,times
0,60.13,-1.18,2015-06-22 00:00:00,1434931200,2015-06-22,00:00:00
1,60.13,-1.18,2015-06-22 00:01:00,1434931260,2015-06-22,00:01:00
2,60.13,-1.18,2015-06-22 00:02:00,1434931320,2015-06-22,00:02:00
3,60.13,-1.18,2015-06-22 00:03:00,1434931380,2015-06-22,00:03:00
4,60.13,-1.18,2015-06-22 00:04:00,1434931440,2015-06-22,00:04:00


## Get Unique Dates

In [114]:
datestimeslist = []
for index, row in GPSData.iterrows():
    datetimerow  = row['gpsDateTime']
    daterow = row['dates']
    hourrow = row['times']
    hourrow = hourrow.strftime('%H:%M:%S')
    if hourrow < '04:00:00':
        date_bfr = daterow - (timedelta(days=1))
        datestimeslist.append(daterow)
        datestimeslist.append(date_bfr)
    if hourrow > '20:00:00':
        Date_aft = daterow + (timedelta(days=1))
        datestimeslist.append(daterow)
        datestimeslist.append(Date_aft)  
    else:
        datestimeslist.append(daterow)

In [115]:
def uniquelistdates(list): 
    x = np.array(list) 
    uniquelist = np.unique(x)
    return uniquelist

uniquelist_dates = uniquelistdates(datestimeslist)
uniquelist_dates

array([datetime.date(2015, 6, 21), datetime.date(2015, 6, 22),
       datetime.date(2015, 6, 23), datetime.date(2015, 6, 24)],
      dtype=object)

## Get Residuals from Swarm

In [116]:
# 1. Every Day, Get the Swarm Data and Residuals: Get_Swarm_and_residuals
# Filter the Bad Ones
# Input:  Time variables
# Output: Swarm DF, including the residuals, ABC combined.

def Get_Swarm_residuals(startDateTime, endDateTime):
    requestA = SwarmRequest() 
    requestB = SwarmRequest()
    requestC = SwarmRequest()
    ### Request for Sat A
    requestA.set_collection("SW_OPER_MAGA_LR_1B")
    requestA.set_products(
        measurements=[
            'F', #Magnetic intensity
            'B_NEC', #The Magnetic values are in NEC system (North, East, Centre)

        ],

        models = [
            
            '"CHAOS_MCO_MLI_MMA" = "CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA-Primary" + "CHAOS-MMA-Secondary"'
        ],
        residuals=True, #Brining the residuals.
        sampling_step="PT60S", #Get the data every 60 seconds. 
    )
    
    #Define an array to store the data request for Satellite A, based on the start Date and time.
    #You can display dsA to get an idea of how the data is requested.
    dsA = requestA.get_between(
        start_time=startDateTime,
        end_time=endDateTime
    ).as_dataframe()
           
    ### End Request for Sat A
    
    ### Request for Sat B
    requestB.set_collection("SW_OPER_MAGB_LR_1B")
    requestB.set_products(
        measurements=[
            'F', #Magnetic intensity
            'B_NEC',

        ],
        models = [
            
            '"CHAOS_MCO_MLI_MMA" = "CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA-Primary" + "CHAOS-MMA-Secondary"'
        ],
        residuals=True, 
        sampling_step="PT60S",
    )

    dsB = requestB.get_between(
        start_time= startDateTime,
        end_time= endDateTime
    ).as_dataframe()
    ### End Request for Sat B
    ### Request for Sat C
    requestC.set_collection("SW_OPER_MAGC_LR_1B")
    requestC.set_products(
        measurements=[
            'F',
            'B_NEC',

        ],
        models = [
            
            '"CHAOS_MCO_MLI_MMA" = "CHAOS-Core" + "CHAOS-Static" + "CHAOS-MMA-Primary" + "CHAOS-MMA-Secondary"'
        ],
        residuals=True,
        sampling_step="PT60S",
    )

    dsC = requestC.get_between(
        start_time=startDateTime,
        end_time=endDateTime
    ).as_dataframe()
    ### End Request for Sat C
    dsA[['N_res','E_res','C_res']] = pd.DataFrame(dsA.B_NEC_res_CHAOS_MCO_MLI_MMA.tolist(), index= dsA.index)
    dsB[['N_res','E_res','C_res']] = pd.DataFrame(dsB.B_NEC_res_CHAOS_MCO_MLI_MMA.tolist(), index= dsB.index)
    dsC[['N_res','E_res','C_res']] = pd.DataFrame(dsC.B_NEC_res_CHAOS_MCO_MLI_MMA.tolist(), index= dsC.index)
    dsA_filtered = dsA[dsA['F_res_CHAOS_MCO_MLI_MMA'].between(-2000, 2000)]
    dsB_filtered = dsB[dsB['F_res_CHAOS_MCO_MLI_MMA'].between(-2000, 2000)]
    dsC_filtered = dsC[dsA['F_res_CHAOS_MCO_MLI_MMA'].between(-2000, 2000)]

    dsA_filtered['epoch'] = dsA_filtered.index
    dsA_filtered['timestamp'] = dsA_filtered.index
    dsA_filtered['epoch'] = dsA_filtered['epoch'].astype('int64')//1e9
    dsA_filtered['epoch'] = dsA_filtered['epoch'].astype(int)
    dsA_filtered.set_index("epoch", inplace=True)
    
    dsB_filtered['epoch'] = dsB_filtered.index
    dsB_filtered['timestamp'] = dsB_filtered.index
    dsB_filtered['epoch'] = dsB_filtered['epoch'].astype('int64')//1e9
    dsB_filtered['epoch'] = dsB_filtered['epoch'].astype(int)
    dsB_filtered.set_index("epoch", inplace=True)
    
    dsC_filtered['epoch'] = dsC_filtered.index
    dsC_filtered['timestamp'] = dsC_filtered.index
    dsC_filtered['epoch'] = dsC_filtered['epoch'].astype('int64')//1e9
    dsC_filtered['epoch'] = dsC_filtered['epoch'].astype(int)
    dsC_filtered.set_index("epoch", inplace=True)

    return dsA_filtered, dsB_filtered, dsC_filtered

In [117]:
%%time

hours_t_day = 24
hours_added = datetime.timedelta(hours = hours_t_day)

listdfa = []
listdfb = []
listdfc = []

cc = ClientConfig()
cc.set_site_config("https://vires.services/ows", token=VirESToken)
cc.default_url = "https://vires.services/ows"
cc.save()

for d in uniquelist_dates:
    print("Process for:",d,":" )
    startdate = datetime.datetime.combine(d, datetime.datetime.min.time())
    enddate = startdate + hours_added
    SwarmResidualsA,SwarmResidualsB,SwarmResidualsC = Get_Swarm_residuals(startdate, enddate)
    listdfa.append(SwarmResidualsA)
    listdfb.append(SwarmResidualsB)
    listdfc.append(SwarmResidualsC)

Process for: 2015-06-21 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:04, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Process for: 2015-06-22 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:04, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:04, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Process for: 2015-06-23 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Process for: 2015-06-24 :


[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)
[1/1] Processing:  100%|████████████████████████████████████████████████████████|  [ Elapsed: 00:02, Remaining: 00:00 ]
      Downloading: 100%|██████████████████████████████████████████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.106MB)


Wall time: 53.2 s


In [118]:
TotalSwarmRes_A = pd.concat(listdfa, join='outer', axis=0)
TotalSwarmRes_B = pd.concat(listdfb, join='outer', axis=0)
TotalSwarmRes_C = pd.concat(listdfc, join='outer', axis=0)
TotalSwarmRes_C

,Longitude,B_NEC_res_CHAOS_MCO_MLI_MMA,Radius,Latitude,Spacecraft,F_res_CHAOS_MCO_MLI_MMA,N_res,E_res,C_res,timestamp
epoch,,,,,,,,,,
1434844800,175.242936,"[1.4274562389864514, -2.7902306064056006, -7.0...",6821115.83,71.515267,C,-46898.032569,1.427456,-2.790231,-7.093420,2015-06-21 00:00:00
1434844860,177.205998,"[4.747213080770962, -6.6550294747679, -6.82272...",6820850.66,75.315679,C,-47286.789355,4.747213,-6.655029,-6.822724,2015-06-21 00:01:00
1434844920,-179.337410,"[30.28011110464604, -35.25501865900583, -5.706...",6820649.78,79.081011,C,-47429.458164,30.280111,-35.255019,-5.706748,2015-06-21 00:02:00
1434844980,-172.118756,"[-130.71396062631516, 319.33461710013944, -11....",6820515.31,82.756165,C,-47351.528985,-130.713961,319.334617,-11.050118,2015-06-21 00:03:00
1434845040,-151.264836,"[-199.76523745991403, -77.29849782586274, -68....",6820448.68,86.078593,C,-47110.685466,-199.765237,-77.298498,-68.961069,2015-06-21 00:04:00
...,...,...,...,...,...,...,...,...,...,...
1435190100,-13.838505,"[-4.806242421058414, 1.5121612121884027, -0.86...",6837151.56,-48.606684,C,-22102.359033,-4.806242,1.512161,-0.868909,2015-06-24 23:55:00
1435190160,-13.649920,"[-4.684596738718028, 1.5415229067170912, -0.89...",6837309.21,-52.431881,C,-23077.950396,-4.684597,1.541523,-0.899471,2015-06-24 23:56:00
1435190220,-13.377753,"[-3.5795241637551953, 1.8758158927671502, -0.7...",6837442.96,-56.254627,C,-24382.302577,-3.579524,1.875816,-0.775706,2015-06-24 23:57:00


## ST-IDW Process for residuals

In [119]:
# 2. Filter Space and time ST-IDW based on GPS points. ST_IDW_Process
# Interpolation of the Swarm Residuals., NEC interpolated residuals for each GPS Point.

# Input:  GPS Track columns, SwarmDataDF
# Output: GPSTrack+ResidualsInterpolated

def ST_IDW_Process (GPSLat,GPSLong,GPSDateTime,GPSTime, DT):
    
    # 1. Runnig the DfTime_func function to filter by the defined Delta Time.
    time_kernel_A = DfTime_func(TotalSwarmRes_A,GPSTime,DT)
    time_kernel_B = DfTime_func(TotalSwarmRes_B,GPSTime,DT)
    time_kernel_C = DfTime_func(TotalSwarmRes_C,GPSTime,DT)
    #2. Computing the dt as the difference between the datetime and the datetime from swarm point. At this point
    #   we have filtered the swarm point by time.
    time_kernel_A['dT'] = (GPSTime - (time_kernel_A.index))
    time_kernel_B['dT'] = (GPSTime - (time_kernel_B.index))
    time_kernel_C['dT'] = (GPSTime - (time_kernel_C.index))
    
    #3.Computing the ds
    # 3.1 Parsing the requieres parameters for distance_to_GPS function
    s_lat = GPSLat; e_lat = time_kernel_A['Latitude']; s_lng = GPSLong; e_lng = time_kernel_A['Longitude']
    
    ## Running the function, based on the previous parameters, for SatA
    time_kernel_A['distance']= distance_to_GPS(s_lat, s_lng, e_lat, e_lng) 
    
    s_lat = GPSLat; e_lat = time_kernel_B['Latitude']; s_lng = GPSLong; e_lng = time_kernel_B['Longitude']  
    # Running the function, based on the previous parameters, for SatB
    time_kernel_B['distance']= distance_to_GPS(s_lat, s_lng, e_lat, e_lng)
    
    s_lat = GPSLat; e_lat = time_kernel_C['Latitude']; s_lng = GPSLong; e_lng = time_kernel_C['Longitude']  
    # Running the function, based on the previous parameters, for SatC
    time_kernel_C['distance']= distance_to_GPS(s_lat, s_lng, e_lat, e_lng) 
    
    #3.2 Computing the R distance.
    time_kernel_A['r']= Kradius(GPSLat)
    time_kernel_B['r']= Kradius(GPSLat)
    time_kernel_C['r']= Kradius(GPSLat)
    
    #3.3 Filtering row that only fall into the computed R value.
    space_time_kA=time_kernel_A[time_kernel_A['distance']<=time_kernel_A['r']]
    space_time_kB=time_kernel_B[time_kernel_B['distance']<=time_kernel_B['r']]
    space_time_kC=time_kernel_C[time_kernel_C['distance']<=time_kernel_C['r']]
    
    #4. Calculating the number of points per satellite that have passed the Space and Time Windows.
    NumPointsA = len(space_time_kA.index)
    NumPointsB = len(space_time_kB.index)
    NumPointsC = len(space_time_kC.index)
    TolSatPts = (NumPointsA+NumPointsB+NumPointsC)
    
    #7. Combining the three satellited messures into a bigger dataframe that store all the Swarm points that were filtered. 
    frames = [space_time_kA, space_time_kB, space_time_kC] #List to index the specific SatId to the new full DF.
    SwarmResiduals_ST_filtered = pd.concat(frames, keys=['A', 'B', 'C'], sort=False)
    
    #8. Computing the minimum distance and the average distance.
    MinDistance = SwarmResiduals_ST_filtered['distance'].min()
    AvDistance = SwarmResiduals_ST_filtered['distance'].mean()
    
    #9. Computing the d (hypotenuse compused from the edges ds, dt values
    ds = SwarmResiduals_ST_filtered['distance']
    r = SwarmResiduals_ST_filtered['r']
    dt = SwarmResiduals_ST_filtered['dT']
    SwarmResiduals_ST_filtered['Dj']= DistJ(ds, r, dt, DT)
   
    #10 Calculating the weigth values based on the previuos parameters.
    SwarmResiduals_ST_filtered['W']= 1/((SwarmResiduals_ST_filtered['Dj'])**2) #We need to make more clear this part.
    
    #11. Computing the Sum of weigths
    SumW = SwarmResiduals_ST_filtered['W'].sum()
    #12. Distribution of weigths
    SwarmResiduals_ST_filtered['Wj'] = SwarmResiduals_ST_filtered['W']/SumW 
    #Could be  some bias. So we need to find a way to apply this weigth maybe base on distance and time using DT (4hrs)
    
    #13. Computing the Magnetic componente based on the weigths prevoius weigths. 
    N_res_int = (SwarmResiduals_ST_filtered['Wj']*SwarmResiduals_ST_filtered['N_res']).sum()
    E_res_int = (SwarmResiduals_ST_filtered['Wj']*SwarmResiduals_ST_filtered['E_res']).sum()
    C_res_int = (SwarmResiduals_ST_filtered['Wj']*SwarmResiduals_ST_filtered['C_res']).sum()

    #15. Write the results into an array that will be a dictionay for the final dataframe.
    resultrowGPS = {'Latitude': GPSLat, 'Longitude': GPSLong, 'DateTime': GPSDateTime, 'N_res': N_res_int, 'E_res': E_res_int, 'C_res':C_res_int, 'TotalPoints':TolSatPts, 'Minimum_Distance':MinDistance, 'Average_Distance':AvDistance}  
    return resultrowGPS

In [120]:
def distance_to_GPS(s_lat, s_lng, e_lat, e_lng): 
    # approximate radius of earth in km
    R = 6373.0
    s_lat = s_lat*(np.pi)/180.0                     
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    return 2 * R * np.arcsin(np.sqrt(d))

def Kradius (lat):
    if 0 <= lat < 90 :
        #for Northern Latitudes 
        nlat = (-10 * lat) + 1800
        #print("The R on the North")
    if -90 < lat < 0:
        #for Southern Latitudes
        nlat = (10 * lat) + 1800
        #print ("The R on the South")
    return nlat

def DistJ(ds, r, dt, DT):
    eDist = np.sqrt(((ds/r)**2 + (dt/DT)**2)/2)
    return eDist

def DfTime_func (SwarmData, GPSTime, DT):
#     DT= 14400 #deltaT of 14400 secs, 4 hours, if the Period is 1s. 1440 for 10s of period. 240 for data every 60s.
    DataFrame_Per_Time = []
    for index in SwarmData.index:
        if index == GPSTime:
            DataFrame_Per_Time = pd.DataFrame(SwarmData.loc[index-DT:index+DT])
    return DataFrame_Per_Time

In [121]:
%%time
#Sequential mode, applying a traditional loop using iterrows.
DT= 14400
if __name__ == '__main__':
    dn = [] ## List used to add all the GPS points with the annotated MAG Data. See the last bullet point of this process        
    for index, row in GPSData.iterrows():
        GPSLat = row['gpsLat']
        GPSLong = row['gpsLong']
        GPSDateTime = row['gpsDateTime']
        GPSTime = row['epoch']
        print("Process for:", index,"DateTime:",GPSDateTime)
        try:
            result=ST_IDW_Process(GPSLat,GPSLong,GPSDateTime,GPSTime, DT)
            dn.append(result)
        except:
            print("Ups!.That was a bad Swarm Point, let's keep working with the next point")
            result_badPoint= {'Latitude': GPSLat, 'Longitude': GPSLong, 'DateTime': GPSDateTime, 'N_res': 0, 'E_res': 0, 'C_res':0, 'TotalPoints':0, 'Minimum_Distance':0, 'Average_Distance':0}  
            dn.append(result_badPoint)
            continue

Process for: 0 DateTime: 2015-06-22 00:00:00
Process for: 1 DateTime: 2015-06-22 00:01:00
Process for: 2 DateTime: 2015-06-22 00:02:00
Process for: 3 DateTime: 2015-06-22 00:03:00
Process for: 4 DateTime: 2015-06-22 00:04:00
Process for: 5 DateTime: 2015-06-22 00:05:00
Process for: 6 DateTime: 2015-06-22 00:06:00
Process for: 7 DateTime: 2015-06-22 00:07:00
Process for: 8 DateTime: 2015-06-22 00:08:00
Process for: 9 DateTime: 2015-06-22 00:09:00
Process for: 10 DateTime: 2015-06-22 00:10:00
Process for: 11 DateTime: 2015-06-22 00:11:00
Process for: 12 DateTime: 2015-06-22 00:12:00
Process for: 13 DateTime: 2015-06-22 00:13:00
Process for: 14 DateTime: 2015-06-22 00:14:00
Process for: 15 DateTime: 2015-06-22 00:15:00
Process for: 16 DateTime: 2015-06-22 00:16:00
Process for: 17 DateTime: 2015-06-22 00:17:00
Process for: 18 DateTime: 2015-06-22 00:18:00
Process for: 19 DateTime: 2015-06-22 00:19:00
Process for: 20 DateTime: 2015-06-22 00:20:00
Process for: 21 DateTime: 2015-06-22 00:21:0

Process for: 178 DateTime: 2015-06-22 02:58:00
Process for: 179 DateTime: 2015-06-22 02:59:00
Process for: 180 DateTime: 2015-06-22 03:00:00
Process for: 181 DateTime: 2015-06-22 03:01:00
Process for: 182 DateTime: 2015-06-22 03:02:00
Process for: 183 DateTime: 2015-06-22 03:03:00
Process for: 184 DateTime: 2015-06-22 03:04:00
Process for: 185 DateTime: 2015-06-22 03:05:00
Process for: 186 DateTime: 2015-06-22 03:06:00
Process for: 187 DateTime: 2015-06-22 03:07:00
Process for: 188 DateTime: 2015-06-22 03:08:00
Process for: 189 DateTime: 2015-06-22 03:09:00
Process for: 190 DateTime: 2015-06-22 03:10:00
Process for: 191 DateTime: 2015-06-22 03:11:00
Process for: 192 DateTime: 2015-06-22 03:12:00
Process for: 193 DateTime: 2015-06-22 03:13:00
Process for: 194 DateTime: 2015-06-22 03:14:00
Process for: 195 DateTime: 2015-06-22 03:15:00
Process for: 196 DateTime: 2015-06-22 03:16:00
Process for: 197 DateTime: 2015-06-22 03:17:00
Process for: 198 DateTime: 2015-06-22 03:18:00
Process for: 

Process for: 357 DateTime: 2015-06-22 05:57:00
Process for: 358 DateTime: 2015-06-22 05:58:00
Process for: 359 DateTime: 2015-06-22 05:59:00
Process for: 360 DateTime: 2015-06-22 06:00:00
Process for: 361 DateTime: 2015-06-22 06:01:00
Process for: 362 DateTime: 2015-06-22 06:02:00
Process for: 363 DateTime: 2015-06-22 06:03:00
Process for: 364 DateTime: 2015-06-22 06:04:00
Process for: 365 DateTime: 2015-06-22 06:05:00
Process for: 366 DateTime: 2015-06-22 06:06:00
Process for: 367 DateTime: 2015-06-22 06:07:00
Process for: 368 DateTime: 2015-06-22 06:08:00
Process for: 369 DateTime: 2015-06-22 06:09:00
Process for: 370 DateTime: 2015-06-22 06:10:00
Process for: 371 DateTime: 2015-06-22 06:11:00
Process for: 372 DateTime: 2015-06-22 06:12:00
Process for: 373 DateTime: 2015-06-22 06:13:00
Process for: 374 DateTime: 2015-06-22 06:14:00
Process for: 375 DateTime: 2015-06-22 06:15:00
Process for: 376 DateTime: 2015-06-22 06:16:00
Process for: 377 DateTime: 2015-06-22 06:17:00
Process for: 

Process for: 532 DateTime: 2015-06-22 08:52:00
Process for: 533 DateTime: 2015-06-22 08:53:00
Process for: 534 DateTime: 2015-06-22 08:54:00
Process for: 535 DateTime: 2015-06-22 08:55:00
Process for: 536 DateTime: 2015-06-22 08:56:00
Process for: 537 DateTime: 2015-06-22 08:57:00
Process for: 538 DateTime: 2015-06-22 08:58:00
Process for: 539 DateTime: 2015-06-22 08:59:00
Process for: 540 DateTime: 2015-06-22 09:00:00
Process for: 541 DateTime: 2015-06-22 09:01:00
Process for: 542 DateTime: 2015-06-22 09:02:00
Process for: 543 DateTime: 2015-06-22 09:03:00
Process for: 544 DateTime: 2015-06-22 09:04:00
Process for: 545 DateTime: 2015-06-22 09:05:00
Process for: 546 DateTime: 2015-06-22 09:06:00
Process for: 547 DateTime: 2015-06-22 09:07:00
Process for: 548 DateTime: 2015-06-22 09:08:00
Process for: 549 DateTime: 2015-06-22 09:09:00
Process for: 550 DateTime: 2015-06-22 09:10:00
Process for: 551 DateTime: 2015-06-22 09:11:00
Process for: 552 DateTime: 2015-06-22 09:12:00
Process for: 

Process for: 712 DateTime: 2015-06-22 11:52:00
Process for: 713 DateTime: 2015-06-22 11:53:00
Process for: 714 DateTime: 2015-06-22 11:54:00
Process for: 715 DateTime: 2015-06-22 11:55:00
Process for: 716 DateTime: 2015-06-22 11:56:00
Process for: 717 DateTime: 2015-06-22 11:57:00
Process for: 718 DateTime: 2015-06-22 11:58:00
Process for: 719 DateTime: 2015-06-22 11:59:00
Process for: 720 DateTime: 2015-06-22 12:00:00
Process for: 721 DateTime: 2015-06-22 12:01:00
Process for: 722 DateTime: 2015-06-22 12:02:00
Process for: 723 DateTime: 2015-06-22 12:03:00
Process for: 724 DateTime: 2015-06-22 12:04:00
Process for: 725 DateTime: 2015-06-22 12:05:00
Process for: 726 DateTime: 2015-06-22 12:06:00
Process for: 727 DateTime: 2015-06-22 12:07:00
Process for: 728 DateTime: 2015-06-22 12:08:00
Process for: 729 DateTime: 2015-06-22 12:09:00
Process for: 730 DateTime: 2015-06-22 12:10:00
Process for: 731 DateTime: 2015-06-22 12:11:00
Process for: 732 DateTime: 2015-06-22 12:12:00
Process for: 

...

Process for: 2100 DateTime: 2015-06-23 11:00:00
Process for: 2101 DateTime: 2015-06-23 11:01:00
Process for: 2102 DateTime: 2015-06-23 11:02:00
Process for: 2103 DateTime: 2015-06-23 11:03:00
Process for: 2104 DateTime: 2015-06-23 11:04:00
Process for: 2105 DateTime: 2015-06-23 11:05:00
Process for: 2106 DateTime: 2015-06-23 11:06:00
Process for: 2107 DateTime: 2015-06-23 11:07:00
Process for: 2108 DateTime: 2015-06-23 11:08:00
Process for: 2109 DateTime: 2015-06-23 11:09:00
Process for: 2110 DateTime: 2015-06-23 11:10:00
Process for: 2111 DateTime: 2015-06-23 11:11:00
Process for: 2112 DateTime: 2015-06-23 11:12:00
Process for: 2113 DateTime: 2015-06-23 11:13:00
Process for: 2114 DateTime: 2015-06-23 11:14:00
Process for: 2115 DateTime: 2015-06-23 11:15:00
Process for: 2116 DateTime: 2015-06-23 11:16:00
Process for: 2117 DateTime: 2015-06-23 11:17:00
Process for: 2118 DateTime: 2015-06-23 11:18:00
Process for: 2119 DateTime: 2015-06-23 11:19:00
Process for: 2120 DateTime: 2015-06-23 1

Process for: 2272 DateTime: 2015-06-23 13:52:00
Process for: 2273 DateTime: 2015-06-23 13:53:00
Process for: 2274 DateTime: 2015-06-23 13:54:00
Process for: 2275 DateTime: 2015-06-23 13:55:00
Process for: 2276 DateTime: 2015-06-23 13:56:00
Process for: 2277 DateTime: 2015-06-23 13:57:00
Process for: 2278 DateTime: 2015-06-23 13:58:00
Process for: 2279 DateTime: 2015-06-23 13:59:00
Process for: 2280 DateTime: 2015-06-23 14:00:00
Process for: 2281 DateTime: 2015-06-23 14:01:00
Process for: 2282 DateTime: 2015-06-23 14:02:00
Process for: 2283 DateTime: 2015-06-23 14:03:00
Process for: 2284 DateTime: 2015-06-23 14:04:00
Process for: 2285 DateTime: 2015-06-23 14:05:00
Process for: 2286 DateTime: 2015-06-23 14:06:00
Process for: 2287 DateTime: 2015-06-23 14:07:00
Process for: 2288 DateTime: 2015-06-23 14:08:00
Process for: 2289 DateTime: 2015-06-23 14:09:00
Process for: 2290 DateTime: 2015-06-23 14:10:00
Process for: 2291 DateTime: 2015-06-23 14:11:00
Process for: 2292 DateTime: 2015-06-23 1

Process for: 2447 DateTime: 2015-06-23 16:47:00
Process for: 2448 DateTime: 2015-06-23 16:48:00
Process for: 2449 DateTime: 2015-06-23 16:49:00
Process for: 2450 DateTime: 2015-06-23 16:50:00
Process for: 2451 DateTime: 2015-06-23 16:51:00
Process for: 2452 DateTime: 2015-06-23 16:52:00
Process for: 2453 DateTime: 2015-06-23 16:53:00
Process for: 2454 DateTime: 2015-06-23 16:54:00
Process for: 2455 DateTime: 2015-06-23 16:55:00
Process for: 2456 DateTime: 2015-06-23 16:56:00
Process for: 2457 DateTime: 2015-06-23 16:57:00
Process for: 2458 DateTime: 2015-06-23 16:58:00
Process for: 2459 DateTime: 2015-06-23 16:59:00
Process for: 2460 DateTime: 2015-06-23 17:00:00
Process for: 2461 DateTime: 2015-06-23 17:01:00
Process for: 2462 DateTime: 2015-06-23 17:02:00
Process for: 2463 DateTime: 2015-06-23 17:03:00
Process for: 2464 DateTime: 2015-06-23 17:04:00
Process for: 2465 DateTime: 2015-06-23 17:05:00
Process for: 2466 DateTime: 2015-06-23 17:06:00
Process for: 2467 DateTime: 2015-06-23 1

Process for: 2619 DateTime: 2015-06-23 19:39:00
Process for: 2620 DateTime: 2015-06-23 19:40:00
Process for: 2621 DateTime: 2015-06-23 19:41:00
Process for: 2622 DateTime: 2015-06-23 19:42:00
Process for: 2623 DateTime: 2015-06-23 19:43:00
Process for: 2624 DateTime: 2015-06-23 19:44:00
Process for: 2625 DateTime: 2015-06-23 19:45:00
Process for: 2626 DateTime: 2015-06-23 19:46:00
Process for: 2627 DateTime: 2015-06-23 19:47:00
Process for: 2628 DateTime: 2015-06-23 19:48:00
Process for: 2629 DateTime: 2015-06-23 19:49:00
Process for: 2630 DateTime: 2015-06-23 19:50:00
Process for: 2631 DateTime: 2015-06-23 19:51:00
Process for: 2632 DateTime: 2015-06-23 19:52:00
Process for: 2633 DateTime: 2015-06-23 19:53:00
Process for: 2634 DateTime: 2015-06-23 19:54:00
Process for: 2635 DateTime: 2015-06-23 19:55:00
Process for: 2636 DateTime: 2015-06-23 19:56:00
Process for: 2637 DateTime: 2015-06-23 19:57:00
Process for: 2638 DateTime: 2015-06-23 19:58:00
Process for: 2639 DateTime: 2015-06-23 1

Process for: 2793 DateTime: 2015-06-23 22:33:00
Process for: 2794 DateTime: 2015-06-23 22:34:00
Process for: 2795 DateTime: 2015-06-23 22:35:00
Process for: 2796 DateTime: 2015-06-23 22:36:00
Process for: 2797 DateTime: 2015-06-23 22:37:00
Process for: 2798 DateTime: 2015-06-23 22:38:00
Process for: 2799 DateTime: 2015-06-23 22:39:00
Process for: 2800 DateTime: 2015-06-23 22:40:00
Process for: 2801 DateTime: 2015-06-23 22:41:00
Process for: 2802 DateTime: 2015-06-23 22:42:00
Process for: 2803 DateTime: 2015-06-23 22:43:00
Process for: 2804 DateTime: 2015-06-23 22:44:00
Process for: 2805 DateTime: 2015-06-23 22:45:00
Process for: 2806 DateTime: 2015-06-23 22:46:00
Process for: 2807 DateTime: 2015-06-23 22:47:00
Process for: 2808 DateTime: 2015-06-23 22:48:00
Process for: 2809 DateTime: 2015-06-23 22:49:00
Process for: 2810 DateTime: 2015-06-23 22:50:00
Process for: 2811 DateTime: 2015-06-23 22:51:00
Process for: 2812 DateTime: 2015-06-23 22:52:00
Process for: 2813 DateTime: 2015-06-23 2

In [122]:
GPS_ResInt = pd.DataFrame(dn)
GPS_ResInt

,Latitude,Longitude,DateTime,N_res,E_res,C_res,TotalPoints,Minimum_Distance,Average_Distance
0,60.13,-1.18,2015-06-22 00:00:00,3.758446,3.482282,-0.332932,27,359.356885,835.509839
1,60.13,-1.18,2015-06-22 00:01:00,3.770869,3.422624,-0.333683,27,359.356885,835.509839
2,60.13,-1.18,2015-06-22 00:02:00,3.783418,3.362821,-0.334442,27,359.356885,835.509839
3,60.13,-1.18,2015-06-22 00:03:00,3.796089,3.302881,-0.335209,27,359.356885,835.509839
4,60.13,-1.18,2015-06-22 00:04:00,3.808879,3.242813,-0.335984,27,359.356885,835.509839
...,...,...,...,...,...,...,...,...,...
2875,60.13,-1.18,2015-06-23 23:55:00,55.729992,-64.086637,-31.601461,18,81.730880,644.839875
2876,60.13,-1.18,2015-06-23 23:56:00,56.309381,-64.595955,-32.002550,18,81.730880,644.839875
2877,60.13,-1.18,2015-06-23 23:57:00,56.891924,-65.087598,-32.409269,18,81.730880,644.839875
2878,60.13,-1.18,2015-06-23 23:58:00,57.477308,-65.561640,-32.821344,18,81.730880,644.839875


## CHAOS Ground Values

In [123]:
# 3. Compute the ground CHAOS Values for the previous GPS Track. Ionosphere_correction_results
#    Add the residuals to the computed CHAOS Values.
# Input:  GPSTrack+ResidualsInterpolated
# Output: AnnotatedGPSTrack

In [124]:
def CHAOS_ground_values(GPS_ResInt):
    
    model = load_CHAOS_matfile('C:\\foss4guk19-jupyter-master\\Project_StAndrews\\Parallel\\CHAOS-7.mat')
    theta = 90-GPS_ResInt['Latitude'].values
    phi = GPS_ResInt['Longitude'].values
    rad_geoc_ground, theta_geoc_ground, sd_ground, cd_ground = gg_to_geo(0.380, theta) #PEG altitude in km
    time= mjd2000(pd.DatetimeIndex(GPS_ResInt['DateTime']).year, pd.DatetimeIndex(GPS_ResInt['DateTime']).month, pd.DatetimeIndex(GPS_ResInt['DateTime']).day)

    B_r_core, B_t_core, B_phi_core = model.synth_values_tdep(time, rad_geoc_ground, theta_geoc_ground, phi) #Core Contribution
    B_r_crust, B_t_crust, B_phi_crust = model.synth_values_static(rad_geoc_ground, theta_geoc_ground, phi) #Crust Contribution
    B_r_magneto, B_t_magneto, B_phi_magneto = model.synth_values_gsm(time, rad_geoc_ground, theta_geoc_ground, phi) #Magnetosphere contribution.

    #Change the direcction of the axis from XYZ to r,theta and phi.
    B_r_swarm, B_t_swarm, B_phi_swarm = -GPS_ResInt['C_res'], -GPS_ResInt['N_res'], GPS_ResInt['E_res']


    #Compute the magnetic component (r,theta,phi) at ground level.
    B_r_ground = B_r_core + B_r_crust + B_r_magneto + B_r_swarm #(-Z)
    B_t_ground = B_t_core + B_t_crust + B_t_magneto + B_t_swarm #(-X)
    B_phi_ground = B_phi_core + B_phi_crust +B_phi_magneto + B_phi_swarm #(Y)

    #Convert B_r_, B_t_, and B_phi to XYZ (NEC)
    Z_chaos = -B_r_ground   #Z
    X_chaos = -B_t_ground   #X
    Y_chaos = B_phi_ground  #Y

    #Rotate the X(N) and Z(C) magnetic field values of the chaos models into the geodectic frame using the sd and cd (sine and cosine d from gg_to_geo) 
    X_obs = X_chaos*cd_ground + Z_chaos*sd_ground #New N
    Z_obs = Z_chaos*cd_ground - X_chaos*sd_ground #New C
    Y_obs = Y_chaos # New E
    return X_obs, Y_obs, Z_obs

In [125]:
X_obs, Y_obs, Z_obs =CHAOS_ground_values(GPS_ResInt)
GPS_ResInt['N'] =pd.Series(X_obs)
GPS_ResInt['E'] =pd.Series(Y_obs)
GPS_ResInt['C'] =pd.Series(Z_obs)
GPS_ResInt

,Latitude,Longitude,DateTime,N_res,E_res,C_res,TotalPoints,Minimum_Distance,Average_Distance,N,E,C
0,60.13,-1.18,2015-06-22 00:00:00,3.758446,3.482282,-0.332932,27,359.356885,835.509839,15137.500086,-657.461989,48608.262578
1,60.13,-1.18,2015-06-22 00:01:00,3.770869,3.422624,-0.333683,27,359.356885,835.509839,15137.512507,-657.521647,48608.261791
2,60.13,-1.18,2015-06-22 00:02:00,3.783418,3.362821,-0.334442,27,359.356885,835.509839,15137.525053,-657.581450,48608.260995
3,60.13,-1.18,2015-06-22 00:03:00,3.796089,3.302881,-0.335209,27,359.356885,835.509839,15137.537722,-657.641390,48608.260191
4,60.13,-1.18,2015-06-22 00:04:00,3.808879,3.242813,-0.335984,27,359.356885,835.509839,15137.550509,-657.701458,48608.259379
...,...,...,...,...,...,...,...,...,...,...,...,...
2875,60.13,-1.18,2015-06-23 23:55:00,55.729992,-64.086637,-31.601461,18,81.730880,644.839875,15189.404623,-724.887675,48576.911650
2876,60.13,-1.18,2015-06-23 23:56:00,56.309381,-64.595955,-32.002550,18,81.730880,644.839875,15189.982844,-725.396992,48576.508879
2877,60.13,-1.18,2015-06-23 23:57:00,56.891924,-65.087598,-32.409269,18,81.730880,644.839875,15190.564202,-725.888636,48576.100470
2878,60.13,-1.18,2015-06-23 23:58:00,57.477308,-65.561640,-32.821344,18,81.730880,644.839875,15191.148387,-726.362677,48575.686695


## Compute Magnetic Values based on new NEC Ground Values

In [126]:
#14. Having Intepolated and weigth magnetic values, we can compute the other magnectic components. 
GPS_ResInt['H'] = np.sqrt((GPS_ResInt['N']**2)+(GPS_ResInt['E']**2))
#check the arcgtan in python., From arctan2 is saver.
DgpsRad = np.arctan2(GPS_ResInt['E'],GPS_ResInt['N'])
GPS_ResInt['D'] = np.degrees(DgpsRad)
GPS_ResInt['F'] = np.sqrt((GPS_ResInt['N']**2)+(GPS_ResInt['E']**2)+(GPS_ResInt['C']**2))
IgpsRad = np.arctan2(GPS_ResInt['C'],GPS_ResInt['H'])
GPS_ResInt['I'] = np.degrees(IgpsRad)
GPS_ResInt

,Latitude,Longitude,DateTime,N_res,E_res,C_res,TotalPoints,Minimum_Distance,Average_Distance,N,E,C,H,D,F,I
0,60.13,-1.18,2015-06-22 00:00:00,3.758446,3.482282,-0.332932,27,359.356885,835.509839,15137.500086,-657.461989,48608.262578,15151.771023,-2.486945,50915.020926,72.687125
1,60.13,-1.18,2015-06-22 00:01:00,3.770869,3.422624,-0.333683,27,359.356885,835.509839,15137.512507,-657.521647,48608.261791,15151.786021,-2.487169,50915.024638,72.687109
2,60.13,-1.18,2015-06-22 00:02:00,3.783418,3.362821,-0.334442,27,359.356885,835.509839,15137.525053,-657.581450,48608.260995,15151.801150,-2.487393,50915.028381,72.687092
3,60.13,-1.18,2015-06-22 00:03:00,3.796089,3.302881,-0.335209,27,359.356885,835.509839,15137.537722,-657.641390,48608.260191,15151.816408,-2.487617,50915.032154,72.687076
4,60.13,-1.18,2015-06-22 00:04:00,3.808879,3.242813,-0.335984,27,359.356885,835.509839,15137.550509,-657.701458,48608.259379,15151.831791,-2.487842,50915.035956,72.687059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,60.13,-1.18,2015-06-23 23:55:00,55.729992,-64.086637,-31.601461,18,81.730880,644.839875,15189.404623,-724.887675,48576.911650,15206.691782,-2.732268,50901.471692,72.617604
2876,60.13,-1.18,2015-06-23 23:56:00,56.309381,-64.595955,-32.002550,18,81.730880,644.839875,15189.982844,-725.396992,48576.508879,15207.293632,-2.734080,50901.267121,72.616822
2877,60.13,-1.18,2015-06-23 23:57:00,56.891924,-65.087598,-32.409269,18,81.730880,644.839875,15190.564202,-725.888636,48576.100470,15207.897787,-2.735826,50901.057866,72.616036
2878,60.13,-1.18,2015-06-23 23:58:00,57.477308,-65.561640,-32.821344,18,81.730880,644.839875,15191.148387,-726.362677,48575.686695,15208.503938,-2.737505,50900.844099,72.615246


In [127]:
#Exporting the CSV file
os.chdir(r"../results")
outputfile ="GeoMagV2_"+gpsfilename
export_csv = GPS_ResInt.to_csv (outputfile, index = None, header=True)